In [43]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.model_selection import (
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.linear_model import LinearRegression
from datetime import datetime, timedelta

from scipy.stats import norm
from statsmodels.tsa.statespace.sarimax import SARIMAX

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import pmdarima as pm

import matplotlib.pyplot as plt

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

from pmdarima import auto_arima

In [65]:
df = pd.read_csv("data/jenny/new_df2.csv")
df = df.drop(["count"], axis=1)
df.head(3)

,date,month,year,Temp,Flow,Level,october_Flow,november_Flow,december_Temp,january_Temp,feburary_Temp,october_Level,november_Level,rolling_mean_Temp,rolling_std_Temp,rolling_mean_Flow,rolling_std_Flow,rolling_mean_Level,rolling_std_Level
0,2013-09-02,9,2013,19.127273,0.8755,0.5480,7.679129,28.3825,3.965188,4.324905,3.464881,0.795806,1.16645,9.514444,2.539781,8.101333,3.857294,0.8584,0.102316
1,2013-09-03,9,2013,18.045833,0.7905,0.5405,7.679129,28.3825,3.965188,4.324905,3.464881,0.795806,1.16645,9.514444,2.539781,8.101333,3.857294,0.8584,0.102316
2,2013-09-04,9,2013,17.062500,0.7490,0.5365,7.679129,28.3825,3.965188,4.324905,3.464881,0.795806,1.16645,9.514444,2.539781,8.101333,3.857294,0.8584,0.102316


In [66]:
df_salmon = pd.read_csv('data/salmon_concat.csv') 
df_salmon.head() 

,date,count,70.2,cow bay,mainstem fence,skutz,vimy pool,ck,co
0,2014-05-01,7,NaN,NaN,NaN,NaN,NaN,True,False
1,2014-05-02,34,NaN,NaN,NaN,NaN,NaN,True,False
2,2014-05-07,21,NaN,NaN,NaN,NaN,NaN,True,False
3,2014-05-08,136,NaN,NaN,NaN,NaN,NaN,True,False
4,2014-05-13,74,NaN,NaN,NaN,NaN,NaN,True,False


In [68]:
df = df.merge(df_salmon, on="date", how="left")
print(df["count"].min())
df.head(3)

1.0


,date,month,year,Temp,Flow,Level,october_Flow,november_Flow,december_Temp,january_Temp,...,rolling_mean_Level,rolling_std_Level,count,70.2,cow bay,mainstem fence,skutz,vimy pool,ck,co
0,2013-09-02,9,2013,19.127273,0.8755,0.5480,7.679129,28.3825,3.965188,4.324905,...,0.8584,0.102316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-09-03,9,2013,18.045833,0.7905,0.5405,7.679129,28.3825,3.965188,4.324905,...,0.8584,0.102316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-09-04,9,2013,17.062500,0.7490,0.5365,7.679129,28.3825,3.965188,4.324905,...,0.8584,0.102316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [148]:
for quantile in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
      print(df["count"].quantile(quantile))

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
65.30000000000007


In [69]:
df_dates_check = df.dropna(subset=["70.2", "cow bay", "mainstem fence", "skutz", "vimy pool"])
salmon_dates_begin = df_dates_check["date"].min()
salmon_dates_end = df_dates_check["date"].max()
print(salmon_dates_begin, salmon_dates_end)

2021-04-29 2023-06-07


In [70]:
df = df[(df["date"] >= salmon_dates_begin) & (df["date"] <= salmon_dates_end)]
df.shape

(848, 27)

In [71]:
df["count"] = df["count"].fillna(0)
print(df["count"].min())
print(df["count"].value_counts())

0.0
count
0.0      698
1.0        5
2.0        4
22.0       3
33.0       3
        ... 
257.0      1
425.0      1
339.0      1
710.0      1
604.0      1
Name: count, Length: 120, dtype: int64


In [72]:
df = df.fillna(0)

In [73]:
make_binary = lambda row: 1 if row["count"] >= 1 else 0
df["count_binary"] = df.apply(make_binary, axis=1)
df

,date,month,year,Temp,Flow,Level,october_Flow,november_Flow,december_Temp,january_Temp,...,rolling_std_Level,count,70.2,cow bay,mainstem fence,skutz,vimy pool,ck,co,count_binary
2833,2021-04-29,4,2021,12.954167,5.8300,0.7715,8.635000,34.351333,4.033769,4.458339,...,0.044449,2.0,True,False,False,False,False,True,False,1
2834,2021-04-29,4,2021,12.954167,5.8300,0.7715,8.635000,34.351333,4.033769,4.458339,...,0.044449,59.0,True,False,False,False,False,False,True,1
2835,2021-04-29,4,2021,12.954167,5.8300,0.7715,8.635000,34.351333,4.033769,4.458339,...,0.044449,8.0,False,False,True,False,False,False,True,1
2836,2021-04-30,4,2021,13.966667,7.4500,0.8020,8.635000,34.351333,4.033769,4.458339,...,0.041503,16.0,True,False,False,False,False,False,True,1
2837,2021-04-30,4,2021,13.966667,7.4500,0.8020,8.635000,34.351333,4.033769,4.458339,...,0.041503,6.0,False,False,True,False,False,False,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3676,2023-06-05,6,2023,14.283333,1.4915,0.6525,0.996387,3.854583,1.015860,4.861828,...,0.096525,31.0,False,False,True,False,False,False,True,1
3677,2023-06-06,6,2023,17.233333,1.3790,0.6470,0.996387,3.854583,1.015860,4.861828,...,0.093713,590.0,False,False,True,False,False,True,False,1
3678,2023-06-06,6,2023,17.233333,1.3790,0.6470,0.996387,3.854583,1.015860,4.861828,...,0.093713,27.0,False,False,True,False,False,False,True,1
3679,2023-06-07,6,2023,19.375000,1.2960,0.6425,0.996387,3.854583,1.015860,4.861828,...,0.091714,604.0,False,False,True,False,False,True,False,1


In [74]:
df["count_binary"].value_counts()

count_binary
0    698
1    150
Name: count, dtype: int64

In [75]:
df = df.dropna(subset=["70.2", "cow bay", "mainstem fence", "skutz", "vimy pool"])
df.shape

(848, 28)

In [76]:
df

,date,month,year,Temp,Flow,Level,october_Flow,november_Flow,december_Temp,january_Temp,...,rolling_std_Level,count,70.2,cow bay,mainstem fence,skutz,vimy pool,ck,co,count_binary
2833,2021-04-29,4,2021,12.954167,5.8300,0.7715,8.635000,34.351333,4.033769,4.458339,...,0.044449,2.0,True,False,False,False,False,True,False,1
2834,2021-04-29,4,2021,12.954167,5.8300,0.7715,8.635000,34.351333,4.033769,4.458339,...,0.044449,59.0,True,False,False,False,False,False,True,1
2835,2021-04-29,4,2021,12.954167,5.8300,0.7715,8.635000,34.351333,4.033769,4.458339,...,0.044449,8.0,False,False,True,False,False,False,True,1
2836,2021-04-30,4,2021,13.966667,7.4500,0.8020,8.635000,34.351333,4.033769,4.458339,...,0.041503,16.0,True,False,False,False,False,False,True,1
2837,2021-04-30,4,2021,13.966667,7.4500,0.8020,8.635000,34.351333,4.033769,4.458339,...,0.041503,6.0,False,False,True,False,False,False,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3676,2023-06-05,6,2023,14.283333,1.4915,0.6525,0.996387,3.854583,1.015860,4.861828,...,0.096525,31.0,False,False,True,False,False,False,True,1
3677,2023-06-06,6,2023,17.233333,1.3790,0.6470,0.996387,3.854583,1.015860,4.861828,...,0.093713,590.0,False,False,True,False,False,True,False,1
3678,2023-06-06,6,2023,17.233333,1.3790,0.6470,0.996387,3.854583,1.015860,4.861828,...,0.093713,27.0,False,False,True,False,False,False,True,1
3679,2023-06-07,6,2023,19.375000,1.2960,0.6425,0.996387,3.854583,1.015860,4.861828,...,0.091714,604.0,False,False,True,False,False,True,False,1


In [77]:
map_binary = {
      False: 0,
      True: 1
}
columns_map = ["70.2", "cow bay", "mainstem fence", "skutz", "vimy pool", "ck", "co"]

df[columns_map] = df[columns_map].applymap(map_binary.get)
df.head(3)

/var/folders/c5/pphx1c0n3lb12sy85_ybqxqm0000gn/T/ipykernel_60523/2855669432.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[columns_map] = df[columns_map].applymap(map_binary.get)


,date,month,year,Temp,Flow,Level,october_Flow,november_Flow,december_Temp,january_Temp,...,rolling_std_Level,count,70.2,cow bay,mainstem fence,skutz,vimy pool,ck,co,count_binary
2833,2021-04-29,4,2021,12.954167,5.83,0.7715,8.635,34.351333,4.033769,4.458339,...,0.044449,2.0,1,0,0,0,0,1,0,1
2834,2021-04-29,4,2021,12.954167,5.83,0.7715,8.635,34.351333,4.033769,4.458339,...,0.044449,59.0,1,0,0,0,0,0,1,1
2835,2021-04-29,4,2021,12.954167,5.83,0.7715,8.635,34.351333,4.033769,4.458339,...,0.044449,8.0,0,0,1,0,0,0,1,1


In [78]:
df.shape

(848, 28)

In [121]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=123, shuffle=False)
print(train_df.shape, test_df.shape)

X_train = train_df.drop(columns=["count_binary"])
y_train = train_df["count_binary"]

X_test = test_df.drop(columns=["count_binary"])
y_test = test_df["count_binary"]

(678, 28) (170, 28)


In [122]:
df.columns

Index(['date', 'month', 'year', 'Temp', 'Flow', 'Level', 'october_Flow',
       'november_Flow', 'december_Temp', 'january_Temp', 'feburary_Temp',
       'october_Level', 'november_Level', 'rolling_mean_Temp',
       'rolling_std_Temp', 'rolling_mean_Flow', 'rolling_std_Flow',
       'rolling_mean_Level', 'rolling_std_Level', 'count', '70.2', 'cow bay',
       'mainstem fence', 'skutz', 'vimy pool', 'ck', 'co', 'count_binary'],
      dtype='object')

In [123]:
numeric_cols = df.columns.tolist()[6:19]
numeric_cols

['october_Flow',
 'november_Flow',
 'december_Temp',
 'january_Temp',
 'feburary_Temp',
 'october_Level',
 'november_Level',
 'rolling_mean_Temp',
 'rolling_std_Temp',
 'rolling_mean_Flow',
 'rolling_std_Flow',
 'rolling_mean_Level',
 'rolling_std_Level']

In [124]:
non_adjust_cols = df.columns.tolist()[20:-1]
non_adjust_cols

['70.2', 'cow bay', 'mainstem fence', 'skutz', 'vimy pool', 'ck', 'co']

In [125]:
len(numeric_cols)

13

In [126]:
numeric_transformer = make_pipeline(StandardScaler())
drop_cols = ["date","month","Temp","Flow","Level"]
lr_model = LogisticRegression()

In [127]:
preprocessor = make_column_transformer(
    (numeric_transformer, numeric_cols),
    (OneHotEncoder(drop='if_binary', handle_unknown="ignore"), non_adjust_cols),
    ("drop", drop_cols)
)

In [128]:
preprocessor.fit(X_train)
preprocessor.named_transformers_

{'pipeline': Pipeline(steps=[('standardscaler', StandardScaler())]),
 'onehotencoder': OneHotEncoder(drop='if_binary', handle_unknown='ignore'),
 'drop': 'drop',
 'remainder': 'drop'}

In [129]:
lr_pipe = make_pipeline(preprocessor, lr_model)
lr_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['october_Flow',
                                                   'november_Flow',
                                                   'december_Temp',
                                                   'january_Temp',
                                                   'feburary_Temp',
                                                   'october_Level',
                                                   'november_Level',
                                                   'rolling_mean_Temp',
                                                   'rolling_std_Temp',
                                                   'rolling_mean_Flow',
                                                   'rolling_std_Flow',
                                                   'rolling_mean_Level',
                                                   'rolling_std_Level']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'),
                                                  ['70.2', 'cow bay',
                                                   'mainstem fence', 'skutz',
                                                   'vimy pool', 'ck', 'co']),
                                                 ('drop', 'drop',
                                                  ['date', 'month', 'Temp',
                                                   'Flow', 'Level'])])),
                ('logisticregression', LogisticRegression())])

In [130]:
y_train

2833    1
2834    1
2835    1
2836    1
2837    1
       ..
3506    0
3507    0
3508    0
3509    0
3510    0
Name: count_binary, Length: 678, dtype: int64

In [131]:
scores = cross_validate(lr_pipe, X_train, y_train, return_train_score=True)
print(f"Mean test score: {pd.DataFrame(scores)['test_score'].mean()}")
print(f"Mean training score: {pd.DataFrame(scores)['train_score'].mean()}")
pd.DataFrame(scores)

Mean test score: 0.9911764705882353
Mean training score: 1.0


/Users/jennylee/miniforge3/envs/563/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [1, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,fit_time,score_time,test_score,train_score
0,0.040170,0.002026,1.000000,1.0
1,0.053533,0.001997,1.000000,1.0
2,0.047290,0.010583,0.955882,1.0
3,0.026156,0.004216,1.000000,1.0
4,0.020743,0.003757,1.000000,1.0


In [132]:
coefficients = lr_pipe.named_steps['logisticregression'].coef_[0]
print(len(coefficients))
feature_names = numeric_cols + non_adjust_cols
print(len(feature_names))
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
}).sort_values(by='Coefficient', ascending=False).reset_index().drop(columns = ['index'])

coef_df

20
20


,Feature,Coefficient
0,ck,3.611519
1,co,3.541007
2,mainstem fence,3.298804
3,70.2,1.339338
4,rolling_mean_Level,1.246262
5,cow bay,1.202558
6,skutz,0.716719
7,vimy pool,0.595107
8,january_Temp,0.155596
9,december_Temp,0.155596


In [133]:
lr_pipe.predict(X_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1])

In [141]:
test_df

,date,month,year,Temp,Flow,Level,october_Flow,november_Flow,december_Temp,january_Temp,...,rolling_std_Level,count,70.2,cow bay,mainstem fence,skutz,vimy pool,ck,co,count_binary
3511,2023-01-01,1,2023,2.420833,38.250000,1.5385,0.996387,3.854583,1.01586,4.861828,...,0.569283,0.0,0,0,0,0,0,0,0,0
3512,2023-01-02,1,2023,3.695833,26.599999,1.3465,0.996387,3.854583,1.01586,4.861828,...,0.564726,0.0,0,0,0,0,0,0,0,0
3513,2023-01-03,1,2023,2.875000,20.700000,1.2295,0.996387,3.854583,1.01586,4.861828,...,0.558754,0.0,0,0,0,0,0,0,0,0
3514,2023-01-04,1,2023,5.158333,17.400000,1.1520,0.996387,3.854583,1.01586,4.861828,...,0.552262,0.0,0,0,0,0,0,0,0,0
3515,2023-01-05,1,2023,7.050000,15.849999,1.1100,0.996387,3.854583,1.01586,4.861828,...,0.545559,0.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3676,2023-06-05,6,2023,14.283333,1.491500,0.6525,0.996387,3.854583,1.01586,4.861828,...,0.096525,31.0,0,0,1,0,0,0,1,1
3677,2023-06-06,6,2023,17.233333,1.379000,0.6470,0.996387,3.854583,1.01586,4.861828,...,0.093713,590.0,0,0,1,0,0,1,0,1
3678,2023-06-06,6,2023,17.233333,1.379000,0.6470,0.996387,3.854583,1.01586,4.861828,...,0.093713,27.0,0,0,1,0,0,0,1,1
3679,2023-06-07,6,2023,19.375000,1.296000,0.6425,0.996387,3.854583,1.01586,4.861828,...,0.091714,604.0,0,0,1,0,0,1,0,1


In [144]:
comp_df = pd.DataFrame(lr_pipe.predict(X_test), index=test_df.date, columns=["pred"]).reset_index()
comp_df["actual"] = test_df["count_binary"].tolist()
comp_df

,date,pred,actual
0,2023-01-01,0,0
1,2023-01-02,0,0
2,2023-01-03,0,0
3,2023-01-04,0,0
4,2023-01-05,0,0
...,...,...,...
165,2023-06-05,1,1
166,2023-06-06,1,1
167,2023-06-06,1,1
168,2023-06-07,1,1


In [145]:
test_vals = pd.DataFrame(np.zeros((1, 27)), columns=X_train.columns.tolist())
display(test_vals)
lr_pipe.predict(test_vals)

,date,month,year,Temp,Flow,Level,october_Flow,november_Flow,december_Temp,january_Temp,...,rolling_mean_Level,rolling_std_Level,count,70.2,cow bay,mainstem fence,skutz,vimy pool,ck,co
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


array([1])